In [1]:
!pip install torchtext==0.10.0
from google.colab import drive
import torch 

drive.mount('/content/drive')
path_data=("drive/MyDrive/IMDBDataset/IMDB Dataset.csv")
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device :",device)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 19.6 MB/s 
     |████████████████████████████████| 831.4 MB 15 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.13.1
    Uninstalling torchtext-0.13.1:
      Successfully uninstalled torchtext-0.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.
Mounted at /content/drive
Device : cuda


In [2]:
!pip install transformers
#!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 26.8 MB/s 
     |████████████████████████████████| 7.6 MB 57.4 MB/s 
     |████████████████████████████████| 182 kB 75.2 MB/s 


In [3]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 19.0 MB/s 
     |████████████████████████████████| 132 kB 60.9 MB/s 
     |████████████████████████████████| 212 kB 62.7 MB/s 
     |████████████████████████████████| 127 kB 45.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [4]:
import os
from tqdm import tqdm, trange
import random
import time
import pickle

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import pandas as pd
import math
import time
import pprint
pp = pprint.PrettyPrinter()
import numpy as np
from sklearn.decomposition import PCA

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.optim as optim
from torchtext.legacy import data
from torchtext.data.utils import get_tokenizer

import torch.nn as nn

from transformers import AdamW, XLNetTokenizer, XLNetModel, XLNetLMHeadModel, get_scheduler
from transformers import XLNetConfig,AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification
from datasets import Dataset, load_dataset, load_metric

from keras_preprocessing.sequence import pad_sequences
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [5]:
results={}
models_name = ["BERT","XLNET","LSTM","GLOVE"]
for model_name in models_name:
  results[model_name] = {"Train Loss":[],"Train Acc":[],"Val. Loss": [],"Val. Acc":[],"test. Acc":0,"test. loss":0,"Training time":0}


In [6]:
model_max_length=128
batch_size=32
EPOCH_NUMBER=3

In [7]:
raw_datasets = load_dataset('csv', data_files=path_data)
raw_datasets["train"]= raw_datasets["train"].add_column("labels", pd.Series(raw_datasets["train"]["sentiment"]).map({'negative':0,'positive':1}).tolist())
raw_datasets=raw_datasets["train"].train_test_split(0.5)
raw_data_valid = raw_datasets["train"].train_test_split(0.2)
raw_datasets["train"]= raw_data_valid["train"]
raw_datasets["valid"]= raw_data_valid["test"]
raw_datasets

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-b891201d158e1e45/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['review', 'sentiment', 'labels'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['review', 'sentiment', 'labels'],
        num_rows: 25000
    })
    valid: Dataset({
        features: ['review', 'sentiment', 'labels'],
        num_rows: 5000
    })
})

In [8]:
def tokenize_function(example):
    return tokenizer(example["review"], truncation=True)

tokenizer  = AutoTokenizer.from_pretrained("xlnet-base-cased",model_max_length=model_max_length)
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
tokenized_datasets = tokenized_datasets.remove_columns(["review","sentiment"])
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

  0%|          | 0/20 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [9]:
tokenized_datasets["train"]["input_ids"][0]

tensor([  122,  1432,  5570,   263,    48,  6013,    31,   110,     9,    35,
         2788,    36,    34,    24, 12774,   249,    25,  1254,   696,    21,
           30,   192,  1412,    29,   391,     9,    35,   232,    35,   581,
           71,  2024,    28,  1367,  9644,    34,    48,  3892,   491,    72,
           24,  1519,  3054,    20,    18,    17,    12, 17365,  2087, 10109,
          787,    12,     9,    35,    77,   449,    18,   682,    20, 10959,
           17,  9554,    30,    17,  7049,    88,  1329,  1716,    21,  4401,
            9,   476,    19,    52,  1432,   140,   148,    48,  1585,    31,
          110,    29,    35,   569,   145,    24,  6134,    25,    18,   412,
           26,    23,   204,  1840,   690, 29418,  1785,    21,  1151,   472,
           21,   619,    22,   459,   412,    23,  1166,    19,    65,    38,
           24,    92,     9,   609,    17, 23868,  8549,    23,   110,    27,
           29,   102,   274,   104,   995,  2788,     4,     3])

In [10]:
tokenized_datasets["train"]["token_type_ids"][0]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 2])

In [11]:
tokenized_datasets["train"]["attention_mask"][0]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])

In [12]:
train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=batch_size, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["valid"], batch_size=batch_size, collate_fn=data_collator
)
test_dataloader = DataLoader(
    tokenized_datasets["test"], batch_size=batch_size, collate_fn=data_collator
)

In [13]:
model = AutoModelForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2).to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)
lr_scheduler = get_scheduler("linear",optimizer=optimizer,num_warmup_steps=0,num_training_steps=EPOCH_NUMBER * len(train_dataloader),)

Downloading:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'logits_proj.weight', 'sequence_summary.summary.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [14]:
def train(model,iterator,optimizer,criterion, train_glove=False,train_LSTM=False,train_pretrain=False):
  torch.cuda.empty_cache()
  epoch_loss = 0.0
  epoch_acc = 0.0
  
  model.train()
  metric = load_metric("accuracy")

  for batch in iterator:
      optimizer.zero_grad()

      if train_LSTM:

        text,text_lengths = batch.review
        predictions = model(text,text_lengths).squeeze()
        loss = criterion(predictions,batch.sentiment)

        correct = (torch.round(predictions) == batch.sentiment).float() 
        acc = correct.sum() / len(correct)


        epoch_loss += loss.item()
        epoch_acc += acc.item()
        loss.backward()
        optimizer.step()


      if train_glove:

        text,text_lengths = batch.review
        predictions = model(text).squeeze()
        loss = criterion(predictions,batch.sentiment)

        correct = (torch.round(predictions) == batch.sentiment).float() 
        acc = correct.sum() / len(correct)

        epoch_loss += loss.item()
        epoch_acc += acc.item()
        loss.backward()
        optimizer.step()


      if train_pretrain:
        
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        predictions = outputs.logits
        predictions = torch.argmax(predictions, dim=-1)
        metric.add_batch(predictions=predictions, references=batch["labels"])
        epoch_loss += loss

        loss.backward()
        optimizer.step()
        lr_scheduler.step()

  if  train_pretrain:
     return epoch_loss / len(iterator), metric.compute()["accuracy"]

  if not train_pretrain:
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model,iterator,criterion, train_glove=False,train_LSTM=False,train_pretrain=False):

    epoch_loss = 0.0
    epoch_acc = 0.0
    
    # deactivate the dropouts
    model.eval()
    metric = load_metric("accuracy")
    # Sets require_grad flat False
    with torch.no_grad():
        for batch in iterator:
            if train_LSTM:

              text,text_lengths = batch.review
              predictions = model(text,text_lengths).squeeze()
              loss = criterion(predictions,batch.sentiment)

              correct = (torch.round(predictions) == batch.sentiment).float() 
              acc = correct.sum() / len(correct)

              epoch_loss += loss.item()
              epoch_acc += acc.item()

            if train_glove:

              text,text_lengths = batch.review
              predictions = model(text).squeeze()
              loss = criterion(predictions,batch.sentiment)

              correct = (torch.round(predictions) == batch.sentiment).float() 
              acc = correct.sum() / len(correct)

              epoch_loss += loss.item()
              epoch_acc += acc.item()

            if train_pretrain:
              
              batch = {k: v.to(device) for k, v in batch.items()}
              outputs = model(**batch)
              loss = outputs.loss
              predictions = outputs.logits
              
              predictions = torch.argmax(predictions, dim=-1)

              lr_scheduler.step()
              metric.add_batch(predictions=predictions, references=batch["labels"])




              epoch_loss += loss
    if  train_pretrain:
      return epoch_loss / len(iterator), metric.compute()["accuracy"]
    if not train_pretrain:
      return epoch_loss / len(iterator), epoch_acc / len(iterator)

def plot_results(model_name):
  plt.style.use("seaborn")
  fig, (ax1,ax2) = plt.subplots(2,sharex=True)
  ax1.set_title(model_name,fontsize=18,fontstyle='italic')

  ax1.plot(results[model_name]["Train Acc"],label="Train Acc",linewidth=3)
  ax1.plot(results[model_name]["Val. Acc"],label="Val. Acc",linewidth=3)
  ax1.legend(loc='upper left', shadow=True)
  ax1.set_xlabel('EPOCHS')
  ax1.set_ylabel('Accuracy')

  ax2.plot(results[model_name]["Train Loss"],label="Train Loss",linewidth=3)
  ax2.plot(results[model_name]["Val. Loss"],label="Val. Loss",linewidth=3)
  ax2.legend(loc='upper left', shadow=True)
  ax2.set_xlabel('EPOCHS')
  ax2.set_ylabel('Loss')

  fig.set_figheight(12.3)
  fig.set_figwidth(10)

In [15]:
'''tokenizer  = AutoTokenizer.from_pretrained("bert-base-cased",model_max_length=model_max_length)
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
tokenized_datasets = tokenized_datasets.remove_columns(["review","sentiment"])
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names'''

'tokenizer  = AutoTokenizer.from_pretrained("bert-base-cased",model_max_length=model_max_length)\ntokenized_datasets = raw_datasets.map(tokenize_function, batched=True)\ndata_collator = DataCollatorWithPadding(tokenizer=tokenizer)\ntokenized_datasets = tokenized_datasets.remove_columns(["review","sentiment"])\ntokenized_datasets.set_format("torch")\ntokenized_datasets["train"].column_names'

In [16]:
'''train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=batch_size, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["valid"], batch_size=batch_size, collate_fn=data_collator
)
test_dataloader = DataLoader(
    tokenized_datasets["test"], batch_size=batch_size, collate_fn=data_collator
)'''

'train_dataloader = DataLoader(\n    tokenized_datasets["train"], shuffle=True, batch_size=batch_size, collate_fn=data_collator\n)\neval_dataloader = DataLoader(\n    tokenized_datasets["valid"], batch_size=batch_size, collate_fn=data_collator\n)\ntest_dataloader = DataLoader(\n    tokenized_datasets["test"], batch_size=batch_size, collate_fn=data_collator\n)'

In [17]:
'''model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2).to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)
lr_scheduler = get_scheduler("linear",optimizer=optimizer,num_warmup_steps=0,num_training_steps=EPOCH_NUMBER * len(train_dataloader))'''

'model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2).to(device)\noptimizer = AdamW(model.parameters(), lr=5e-5)\nlr_scheduler = get_scheduler("linear",optimizer=optimizer,num_warmup_steps=0,num_training_steps=EPOCH_NUMBER * len(train_dataloader))'

In [18]:
'''start=time.time()
for epoch in range(1,EPOCH_NUMBER+1):
    print(f"\t Epoch: {epoch}")
    train_loss,train_acc = train(model,train_dataloader,optimizer,None,train_pretrain=True)
    valid_loss,valid_acc = evaluate(model,eval_dataloader,None,train_pretrain=True)
    
    results["BERT"]["Train Loss"].append(train_loss.cpu().detach().numpy())
    results["BERT"]["Train Acc"].append(train_acc*100)
    results["BERT"]["Val. Loss"].append(valid_loss.cpu().detach().numpy())
    results["BERT"]["Val. Acc"].append(valid_acc*100)
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    print()

test_loss,test_acc = evaluate(model,test_dataloader,None,train_pretrain=True)
print(f'\t test. Loss: {test_loss:.3f} |  test. Acc: {test_acc*100:.2f}%')
stop = time.time()
print(f"\t Training time: {stop - start}s")
results["BERT"]["Training time"] = stop - start
results["BERT"]["test. loss"] = test_loss
results["BERT"]["test. Acc"] = test_acc'''


'start=time.time()\nfor epoch in range(1,EPOCH_NUMBER+1):\n    print(f"\t Epoch: {epoch}")\n    train_loss,train_acc = train(model,train_dataloader,optimizer,None,train_pretrain=True)\n    valid_loss,valid_acc = evaluate(model,eval_dataloader,None,train_pretrain=True)\n    \n    results["BERT"]["Train Loss"].append(train_loss.cpu().detach().numpy())\n    results["BERT"]["Train Acc"].append(train_acc*100)\n    results["BERT"]["Val. Loss"].append(valid_loss.cpu().detach().numpy())\n    results["BERT"]["Val. Acc"].append(valid_acc*100)\n    print(f\'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%\')\n    print(f\'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%\')\n    print()\n\ntest_loss,test_acc = evaluate(model,test_dataloader,None,train_pretrain=True)\nprint(f\'\t test. Loss: {test_loss:.3f} |  test. Acc: {test_acc*100:.2f}%\')\nstop = time.time()\nprint(f"\t Training time: {stop - start}s")\nresults["BERT"]["Training time"] = stop - start\nresults["B

In [19]:
def tokenize_function(example):
    return tokenizer(example["review"], truncation=True)

tokenizer  = AutoTokenizer.from_pretrained("xlnet-base-cased",model_max_length=model_max_length)
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
tokenized_datasets = tokenized_datasets.remove_columns(["review","sentiment"])
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

  0%|          | 0/25 [00:00<?, ?ba/s]

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [20]:
train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=batch_size, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["valid"], batch_size=batch_size, collate_fn=data_collator
)
test_dataloader = DataLoader(
    tokenized_datasets["test"], batch_size=batch_size, collate_fn=data_collator
)

In [21]:
model = AutoModelForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2).to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)
lr_scheduler = get_scheduler("linear",optimizer=optimizer,num_warmup_steps=0,num_training_steps=EPOCH_NUMBER * len(train_dataloader),)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'logits_proj.weight', 'sequence_summary.summary.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [22]:
start = time.time()
for epoch in range(1,EPOCH_NUMBER+1):
    print(f"\t Epoch: {epoch}")
    train_loss,train_acc = train(model,train_dataloader,optimizer,None,train_pretrain=True)
    valid_loss,valid_acc = evaluate(model,eval_dataloader,None,train_pretrain=True)
    
    results["XLNET"]["Train Loss"].append(train_loss.cpu().detach().numpy())
    results["XLNET"]["Train Acc"].append(train_acc*100)
    results["XLNET"]["Val. Loss"].append(valid_loss.cpu().detach().numpy())
    results["XLNET"]["Val. Acc"].append(valid_acc*100)
    # Showing statistics
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    print()

test_loss,test_acc = evaluate(model,test_dataloader,None,train_pretrain=True)
print(f'\t test. Loss: {test_loss:.3f} |  test. Acc: {test_acc*100:.2f}%')
stop = time.time()
print(f"\t Training time: {stop - start}s")
results["XLNET"]["Training time"] = stop - start
results["XLNET"]["test. loss"] = test_loss
results["XLNET"]["test. Acc"] = test_acc


	 Epoch: 1


<ipython-input-14-1ae7705c267e>:7: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


You're using a XLNetTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


	Train Loss: 0.321 | Train Acc: 86.08%
	 Val. Loss: 0.260 |  Val. Acc: 89.42%

	 Epoch: 2
	Train Loss: 0.178 | Train Acc: 93.07%
	 Val. Loss: 0.252 |  Val. Acc: 89.92%

	 Epoch: 3
	Train Loss: 0.081 | Train Acc: 97.25%
	 Val. Loss: 0.308 |  Val. Acc: 90.14%

	 test. Loss: 0.321 |  test. Acc: 90.02%
	 Training time: 2430.141296863556s
